In [1]:
import numpy as np

In [114]:
a=np.array([[1,2,3],[2,2,2],[3,3,3],])
b=np.array([[2,2,2],[3,3,3],[1,2,3],])
c=np.array([[3,3,3],[1,2,3],[2,2,2],])
d=np.array([[1,2,3],[3,3,3],[2,2,2],])
lam = np.array([[1,2,1],[1,1,1],[1,2,1],])
gam = np.ones((3,3))
w = np.array([4,4.5,3.5])

def func(x):
    """This on is like their equivariant"""
    # return np.sum(x@( w*np.eye(3) + w*np.ones((3,3)) ) ,axis=0)

    # return np.sum(x@( w*np.ones((3,3)) ) ,axis=0)
    return np.sum(x@( w*np.ones((3,3)) ) ,axis=0)

print(func(a))
print(func(b))
print(func(c))
print(func(d))

[84.  94.5 73.5]
[84.  94.5 73.5]
[84.  94.5 73.5]
[84.  94.5 73.5]


In [102]:
# np.eye(3)*a
# np.ones((1,3)) * np.ones((1,3)).T
# np.ones((1,3)).T * a
w*np.eye(3) + w*np.ones((3,3))

array([[8. , 4.5, 3.5],
       [4. , 9. , 3.5],
       [4. , 4.5, 7. ]])